In [1]:
import os
from PIL import Image
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torch
import torchvision.transforms as transforms
import numpy as np
# import aML.HW2.other.model_structure as model_structure
path = "/Users/liushiwen/Desktop/大四下/hw2/test_data"
path = "D:\Casper\OTHER\Data\MNIST2\HW2_MNIST_train"
dir_list = os.listdir(path)

print(f"Files and directories in '{path}':")

Files and directories in 'D:\Casper\OTHER\Data\MNIST2\HW2_MNIST_train':


0	0.7734375	0.4296875	0.21875	0.21875

(128, 128, 4)


In [2]:
f = open(f"{path}/0000002.txt", "r")
print(f.read())
img = Image.open(f'{path}/0121744.png')
img = np.array(img)
print(img.shape)
img = Image.fromarray(img[:,:,2])
display(img)

In [3]:
img = Image.open(f'{path}/0121700.png')
r, g, b, trash = img.split()

r_array = np.array(r)
g_array = np.array(g)
b_array = np.array(b)

# Check if all channels have the same array values
are_channels_same = np.array_equal(r_array, g_array) and np.array_equal(g_array, b_array)
are_channels_same

True

In [4]:
import os
import pandas as pd
from torch.utils.data import Dataset
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    def __init__(self, data_folder, transform=None):
        self.data_folder = data_folder
        self.image_files = [f for f in os.listdir(data_folder) if f.endswith(".png")]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = os.path.join(self.data_folder, self.image_files[idx])
        image = read_image(image_path)
        image = image[0:3].float()
        # image = image.reshape(1, 128, 128)
        
        txt_file = os.path.splitext(self.image_files[idx])[0] + ".txt"
        txt_path = os.path.join(self.data_folder, txt_file)
        with open(txt_path, "r") as f:
            first_line = f.readline().strip()
            class_label = int(first_line.split()[0])

        if self.transform:
            image = self.transform(image)

        return image, class_label
    
transform = transforms.Compose([])
data_folder = path
custom_dataset = CustomImageDataset(data_folder, transform)


In [5]:
import random
from torch.utils.data import DataLoader, Subset
def get_dataloaders(data_folder, transform, train_ratio, val_ratio, batch_size):
    # Create a single merged dataset
    train_dataset = CustomImageDataset(data_folder, transform)
    val_dataset = CustomImageDataset(data_folder, transform)
    test_dataset = CustomImageDataset(data_folder, transform)

    # obtain training indices that will be used for validation
    num_train = len(test_dataset)
    indices = list(range(num_train))
    print("--------- INDEX checking ---------")
    print(f"Original: {indices[:5]}")
    random.shuffle(indices)
    print(f"Shuffled: {indices[:5]}")
    print("--------- INDEX shuffled ---------\n")

    split_train = int(np.floor(train_ratio * num_train))
    split_val = split_train + int(np.floor(val_ratio * (num_train-split_train)))
    train_idx, val_idx, test_idx = indices[0:split_train], indices[split_train:split_val], indices[split_val:]
    merge_dataset = Subset(train_dataset, train_idx)

    train_loader = DataLoader(merge_dataset, batch_size=batch_size)
    val_loader = DataLoader(Subset(val_dataset, val_idx), batch_size=batch_size)
    test_loader = DataLoader(Subset(test_dataset, test_idx), batch_size=batch_size)
    
    # check dataset
    print(f"Total number of samples: {num_train} datapoints")
    print(f"Number of train samples: {len(train_loader)} batches/ {len(train_loader.dataset)} datapoints")
    print(f"Number of val samples: {len(val_loader)} batches/ {len(val_loader.dataset)} datapoints")
    print(f"Number of test samples: {len(test_loader)} batches/ {len(test_loader.dataset)} datapoints")
    print(f"Data Transform: {transform}")
    print(f"")
    
    dataloaders = {
        "train": train_loader,
        "val": val_loader,
        "test": test_loader,
    }
    return dataloaders

In [6]:
loaders = get_dataloaders(data_folder, transform, 1, 0.5, 64)

--------- INDEX checking ---------
Original: [0, 1, 2, 3, 4]
Shuffled: [21750, 45583, 42951, 36952, 40070]
--------- INDEX shuffled ---------

Total number of samples: 97396 datapoints
Number of train samples: 1522 batches/ 97396 datapoints
Number of val samples: 0 batches/ 0 datapoints
Number of test samples: 0 batches/ 0 datapoints
Data Transform: Compose(
)



In [7]:
def count_parameters(model):
    total_num = 0
    for parameter in model.parameters():
        if parameter.requires_grad:
            total_num += parameter.numel() 
    return total_num


In [8]:
from datetime import datetime

def pprint(output = '\n', show_time = False): # print and fprint at the same time
    filename = "hw2-1-3-MAR24-1.txt"
    print(output)
    with open(filename, 'a') as f:
        if show_time:
            f.write(datetime.now().strftime("[%Y-%m-%d %H:%M:%S] "))

        f.write(str(output))
        f.write('\n')
pprint("OFFICIAL START, Focus on the 1st problem", True)

OFFICIAL START, Focus on the 1st problem


In [11]:
import time
from torch import nn, optim
from tqdm import tqdm

def train(model_lists, model_name):
    model = model_lists[model_name]()
    if 'res' in model_name:
        pprint("modify to 10 with res")
        num_features = model.fc.in_features
        model.fc = torch.nn.Linear(num_features, 10)
    elif 'swin' in model_name:
        pprint("modify to 10 with swin")
        num_features = model.head.in_features
        model.head = torch.nn.Linear(num_features, 10)


    pprint(f"Training model: {model_name}", True)
    model_parameters_amount = count_parameters(model)
    pprint(f"Total parameters: {model_parameters_amount:,}")

    model = model.cuda()
    criterion = nn.CrossEntropyLoss()
    lr= 0.005
    optimizer = optim.Adam(model.parameters(), lr=lr)
    # scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.2)
    pprint(f"Learning rate={lr}")
    iteration = 0
    best_accuracy = 99.75
    epochs = 50
    start = time.time()
    phases = ['train']
    for epoch in range(epochs):
        for phase in phases:
            running_loss = 0.0
            correct_predictions = 0
            correct_top3_predictions = 0
            total_samples = 0
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            for images, labels in tqdm(loaders[phase]): # Iterate over data.
                images, labels = images.cuda(), labels.cuda()
                outputs = model(images)
                loss = criterion(outputs, labels)

                with torch.set_grad_enabled(phase == 'train'):
                    if phase == 'train': # backward + optimize only if in training phase
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                        # scheduler.step()
                running_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct_predictions += (predicted == labels).sum().item()
                
                _, top3_preds = outputs.topk(3, 1, True, True)
                correct_top3_predictions += sum([labels[i] in top3_preds[i] for i in range(labels.size(0))])

                total_samples += labels.size(0)
                iteration += 1

            avg_loss = running_loss / total_samples
            top1_accuracy = correct_predictions / total_samples * 100
            top3_accuracy = correct_top3_predictions / total_samples * 100
            pprint(f"Epoch [{epoch+1}/{epochs}], phase: {phase}, samples: {total_samples}, Loss: {avg_loss:.4f}, Top-1 Accuracy: {top1_accuracy:.2f}%, Top-3 Accuracy: {top3_accuracy:.2f}%")
            if top1_accuracy > best_accuracy:
                best_accuracy = top1_accuracy
                model_scripted = torch.jit.script(model) # Export to TorchScript
                model_scripted.save(f'{model_name}_{best_accuracy}.pt') # Save
                pprint(f"weight saved as: {model_name}_{best_accuracy}.pt")
                
    end = time.time()
    duration = end - start
    pprint(f"Elapsed time: {duration} seconds")
    model_scripted = torch.jit.script(model) # Export to TorchScript
    model_scripted.save(f'{model_name}.pt') # Save
    pprint(f"weight saved as: {model_name}.pt")

In [12]:
import torchvision.models as models

model_lists = {
    'resnet152_hw1': lambda: models.resnet152(weights=models.ResNet152_Weights.DEFAULT),
}

model_names = [
    'resnet152_hw1',
]
for model_name in model_names:
    train(model_lists, model_name)

modify to 10 with res
Training model: resnet152_hw1
Total parameters: 58,164,298
Learning rate=0.005


100%|██████████| 1522/1522 [03:28<00:00,  7.29it/s]


Epoch [1/50], phase: train, samples: 97396, Loss: 0.0048, Top-1 Accuracy: 90.10%, Top-3 Accuracy: 97.49%


100%|██████████| 1522/1522 [03:28<00:00,  7.29it/s]


Epoch [2/50], phase: train, samples: 97396, Loss: 0.0022, Top-1 Accuracy: 95.92%, Top-3 Accuracy: 99.41%


100%|██████████| 1522/1522 [03:29<00:00,  7.25it/s]


Epoch [3/50], phase: train, samples: 97396, Loss: 0.0011, Top-1 Accuracy: 97.86%, Top-3 Accuracy: 99.80%


100%|██████████| 1522/1522 [03:29<00:00,  7.25it/s]


Epoch [4/50], phase: train, samples: 97396, Loss: 0.0009, Top-1 Accuracy: 98.23%, Top-3 Accuracy: 99.86%


100%|██████████| 1522/1522 [03:30<00:00,  7.22it/s]


Epoch [5/50], phase: train, samples: 97396, Loss: 0.0008, Top-1 Accuracy: 98.45%, Top-3 Accuracy: 99.89%


100%|██████████| 1522/1522 [03:32<00:00,  7.15it/s]


Epoch [6/50], phase: train, samples: 97396, Loss: 0.0007, Top-1 Accuracy: 98.62%, Top-3 Accuracy: 99.92%


100%|██████████| 1522/1522 [03:35<00:00,  7.06it/s]


Epoch [7/50], phase: train, samples: 97396, Loss: 0.0006, Top-1 Accuracy: 98.79%, Top-3 Accuracy: 99.93%


100%|██████████| 1522/1522 [03:32<00:00,  7.17it/s]


Epoch [8/50], phase: train, samples: 97396, Loss: 0.0005, Top-1 Accuracy: 98.95%, Top-3 Accuracy: 99.95%


100%|██████████| 1522/1522 [03:32<00:00,  7.17it/s]


Epoch [9/50], phase: train, samples: 97396, Loss: 0.0004, Top-1 Accuracy: 99.12%, Top-3 Accuracy: 99.96%


100%|██████████| 1522/1522 [03:32<00:00,  7.18it/s]


Epoch [10/50], phase: train, samples: 97396, Loss: 0.0004, Top-1 Accuracy: 99.18%, Top-3 Accuracy: 99.97%


100%|██████████| 1522/1522 [03:32<00:00,  7.18it/s]


Epoch [11/50], phase: train, samples: 97396, Loss: 0.0004, Top-1 Accuracy: 99.30%, Top-3 Accuracy: 99.98%


100%|██████████| 1522/1522 [03:32<00:00,  7.17it/s]


Epoch [12/50], phase: train, samples: 97396, Loss: 0.0003, Top-1 Accuracy: 99.32%, Top-3 Accuracy: 99.98%


100%|██████████| 1522/1522 [03:32<00:00,  7.17it/s]


Epoch [13/50], phase: train, samples: 97396, Loss: 0.0003, Top-1 Accuracy: 99.38%, Top-3 Accuracy: 99.99%


100%|██████████| 1522/1522 [03:31<00:00,  7.18it/s]


Epoch [14/50], phase: train, samples: 97396, Loss: 0.0003, Top-1 Accuracy: 99.43%, Top-3 Accuracy: 99.98%


100%|██████████| 1522/1522 [03:32<00:00,  7.18it/s]


Epoch [15/50], phase: train, samples: 97396, Loss: 0.0003, Top-1 Accuracy: 99.48%, Top-3 Accuracy: 99.98%


100%|██████████| 1522/1522 [03:32<00:00,  7.15it/s]


Epoch [16/50], phase: train, samples: 97396, Loss: 0.0002, Top-1 Accuracy: 99.52%, Top-3 Accuracy: 99.99%


100%|██████████| 1522/1522 [03:32<00:00,  7.18it/s]


Epoch [17/50], phase: train, samples: 97396, Loss: 0.0002, Top-1 Accuracy: 99.52%, Top-3 Accuracy: 99.99%


100%|██████████| 1522/1522 [03:31<00:00,  7.18it/s]


Epoch [18/50], phase: train, samples: 97396, Loss: 0.0002, Top-1 Accuracy: 99.62%, Top-3 Accuracy: 99.99%


100%|██████████| 1522/1522 [03:31<00:00,  7.19it/s]


Epoch [19/50], phase: train, samples: 97396, Loss: 0.0002, Top-1 Accuracy: 99.57%, Top-3 Accuracy: 99.99%


100%|██████████| 1522/1522 [03:31<00:00,  7.18it/s]


Epoch [20/50], phase: train, samples: 97396, Loss: 0.0002, Top-1 Accuracy: 99.64%, Top-3 Accuracy: 99.99%


100%|██████████| 1522/1522 [03:32<00:00,  7.17it/s]


Epoch [21/50], phase: train, samples: 97396, Loss: 0.0002, Top-1 Accuracy: 99.67%, Top-3 Accuracy: 99.99%


100%|██████████| 1522/1522 [03:32<00:00,  7.16it/s]


Epoch [22/50], phase: train, samples: 97396, Loss: 0.0002, Top-1 Accuracy: 99.63%, Top-3 Accuracy: 99.99%


100%|██████████| 1522/1522 [03:34<00:00,  7.11it/s]


Epoch [23/50], phase: train, samples: 97396, Loss: 0.0002, Top-1 Accuracy: 99.69%, Top-3 Accuracy: 99.99%


100%|██████████| 1522/1522 [03:34<00:00,  7.11it/s]


Epoch [24/50], phase: train, samples: 97396, Loss: 0.0002, Top-1 Accuracy: 99.70%, Top-3 Accuracy: 99.99%


100%|██████████| 1522/1522 [03:32<00:00,  7.17it/s]


Epoch [25/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.74%, Top-3 Accuracy: 100.00%


100%|██████████| 1522/1522 [03:32<00:00,  7.16it/s]


Epoch [26/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.75%, Top-3 Accuracy: 100.00%


100%|██████████| 1522/1522 [03:32<00:00,  7.16it/s]


Epoch [27/50], phase: train, samples: 97396, Loss: 0.0002, Top-1 Accuracy: 99.70%, Top-3 Accuracy: 100.00%


100%|██████████| 1522/1522 [03:32<00:00,  7.16it/s]


Epoch [28/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.75%, Top-3 Accuracy: 100.00%
weight saved as: resnet152_hw1_99.75255657316522.pt


100%|██████████| 1522/1522 [03:32<00:00,  7.15it/s]


Epoch [29/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.75%, Top-3 Accuracy: 99.99%


100%|██████████| 1522/1522 [03:32<00:00,  7.16it/s]


Epoch [30/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.76%, Top-3 Accuracy: 100.00%
weight saved as: resnet152_hw1_99.7576902542199.pt


100%|██████████| 1522/1522 [03:32<00:00,  7.15it/s]


Epoch [31/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.75%, Top-3 Accuracy: 100.00%


100%|██████████| 1522/1522 [03:32<00:00,  7.16it/s]


Epoch [32/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.76%, Top-3 Accuracy: 100.00%


100%|██████████| 1522/1522 [03:32<00:00,  7.15it/s]


Epoch [33/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.79%, Top-3 Accuracy: 100.00%
weight saved as: resnet152_hw1_99.79362602160253.pt


100%|██████████| 1522/1522 [03:32<00:00,  7.16it/s]


Epoch [34/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.82%, Top-3 Accuracy: 99.99%
weight saved as: resnet152_hw1_99.81621421824305.pt


100%|██████████| 1522/1522 [03:32<00:00,  7.17it/s]


Epoch [35/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.79%, Top-3 Accuracy: 99.99%


100%|██████████| 1522/1522 [03:32<00:00,  7.15it/s]


Epoch [36/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.84%, Top-3 Accuracy: 100.00%
weight saved as: resnet152_hw1_99.8398291510945.pt


100%|██████████| 1522/1522 [03:32<00:00,  7.16it/s]


Epoch [37/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.80%, Top-3 Accuracy: 100.00%


100%|██████████| 1522/1522 [03:32<00:00,  7.16it/s]


Epoch [38/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.77%, Top-3 Accuracy: 100.00%


100%|██████████| 1522/1522 [03:32<00:00,  7.15it/s]


Epoch [39/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.78%, Top-3 Accuracy: 100.00%


100%|██████████| 1522/1522 [03:33<00:00,  7.12it/s]


Epoch [40/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.79%, Top-3 Accuracy: 100.00%


100%|██████████| 1522/1522 [03:34<00:00,  7.10it/s]


Epoch [41/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.87%, Top-3 Accuracy: 100.00%
weight saved as: resnet152_hw1_99.87268470984435.pt


100%|██████████| 1522/1522 [03:32<00:00,  7.18it/s]


Epoch [42/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.82%, Top-3 Accuracy: 100.00%


100%|██████████| 1522/1522 [03:32<00:00,  7.15it/s]


Epoch [43/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.82%, Top-3 Accuracy: 100.00%


100%|██████████| 1522/1522 [03:32<00:00,  7.16it/s]


Epoch [44/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.84%, Top-3 Accuracy: 100.00%


100%|██████████| 1522/1522 [03:32<00:00,  7.16it/s]


Epoch [45/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.86%, Top-3 Accuracy: 100.00%


100%|██████████| 1522/1522 [03:32<00:00,  7.16it/s]


Epoch [46/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.85%, Top-3 Accuracy: 100.00%


100%|██████████| 1522/1522 [03:32<00:00,  7.17it/s]


Epoch [47/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.87%, Top-3 Accuracy: 100.00%


100%|██████████| 1522/1522 [03:32<00:00,  7.16it/s]


Epoch [48/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.86%, Top-3 Accuracy: 100.00%


100%|██████████| 1522/1522 [03:32<00:00,  7.17it/s]


Epoch [49/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.82%, Top-3 Accuracy: 99.99%


100%|██████████| 1522/1522 [03:32<00:00,  7.15it/s]


Epoch [50/50], phase: train, samples: 97396, Loss: 0.0001, Top-1 Accuracy: 99.87%, Top-3 Accuracy: 100.00%
Elapsed time: 10621.786568641663 seconds
weight saved as: resnet152_hw1.pt


In [ ]:
def majority_voting(output_list):
    stacked_outputs = torch.stack(output_list)
    avg_probs = torch.mean(stacked_outputs, dim=0)
    
    return avg_probs

In [ ]:
import time
from torch import nn, optim
from tqdm import tqdm

def eval(model_list):
    for model in model_list:
        model.cuda()
        model.eval()
    criterion = nn.CrossEntropyLoss()
    start = time.time()
    phases = ['val']
    num_class = 10
    for phase in phases:
        running_loss = 0.0
        correct_predictions = 0
        correct_top3_predictions = 0
        total_samples = 0
        confus = torch.zeros(num_class, num_class,dtype=int)            
        for images, labels in tqdm(loaders[phase]): # Iterate over data.
            images, labels = images.cuda(), labels.cuda()
            model_outputs = []
            for model in model_list:
                with torch.no_grad():
                    outputs = model(images)
                    model_outputs.append(outputs)
            vote_outputs = majority_voting(model_outputs)
            loss = criterion(vote_outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(vote_outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            
            _, top3_preds = outputs.topk(3, 1, True, True)
            correct_top3_predictions += sum([labels[i] in top3_preds[i] for i in range(labels.size(0))])
            total_samples += labels.size(0)

            for ii in range(len(predicted)):
                confus[ labels.data[ii] ][ predicted[ii] ]+=1

        avg_loss = running_loss / total_samples
        top1_accuracy = correct_predictions / total_samples * 100
        top3_accuracy = correct_top3_predictions / total_samples * 100
        pprint(f"Phase: {phase}, samples: {total_samples}, Loss: {avg_loss:.4f}, Top-1 Accuracy: {top1_accuracy:.2f}%, Top-3 Accuracy: {top3_accuracy:.2f}%")
        for ii in range(num_class) :
            pprint(f"class {ii}:{confus.numpy()[ii]}")
        # pprint(confus)

    end = time.time()
    duration = end - start
    pprint(f"Elapsed time: {duration} seconds")

In [ ]:
# import torchvision.models as models
# path = "D:\\Casper\\aML\\HW2\\aML\\"
# model_list = [
#     # torch.jit.load(f'{path}R6_btnk_4ch8n4.pt'),
#     torch.jit.load(f'{path}R6_btnk_ch8(hw2).pt'),
#     # torch.jit.load(f'{path}resnet6.pt'),
#     # models.resnet18(weights=True),
#     # models.resnet50(weights=True),
#     # models.resnet101(weights=True),
#     # models.resnet152(weights=True),
#     # models.swin_v2_t(weights=True),
#     # models.swin_v2_s(weights=True),
#     # models.swin_v2_b(weights=True),
#     # models.vit_h_14(weights=models.ViT_H_14_Weights.IMAGENET1K_SWAG_E2E_V1),
# ]

# model_name = [
#     'resnet18',
#     # 'resnet50',
#     # 'resnet101',
#     # 'resnet152',
#     # 'swin_v2_t',
#     # 'swin_v2_s',
#     # 'swin_v2_b',
#     # 'vit_h_14',
# ]
# eval(model_list)

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00,  4.04it/s]

Phase: val, samples: 487, Loss: 0.0006, Top-1 Accuracy: 95.89%, Top-3 Accuracy: 99.59%
class 0:[42  0  0  0  0  0  0  0  0  0]
class 1:[ 1 56  0  0  0  0  0  0  0  0]
class 2:[ 0  0 40  0  0  0  0  0  2  1]
class 3:[ 1  0  0 36  0  0  0  1  0  0]
class 4:[ 0  0  0  0 51  0  0  0  0  1]
class 5:[ 0  1  0  0  0 51  0  0  2  0]
class 6:[ 0  0  0  0  0  0 48  0  0  0]
class 7:[ 1  0  0  0  0  0  0 56  0  2]
class 8:[ 6  0  0  0  0  0  0  0 47  1]
class 9:[ 0  0  0  0  0  0  0  0  0 40]
Elapsed time: 0.4988720417022705 seconds
